# Import libraries and mount drive

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
from pathlib import Path
import re
import requests
from io import StringIO
import random
import pdb

!pip install pandana
import pandana as pdn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Code

In [ ]:
trips = pd.read_csv('/content/drive/MyDrive/Code/Results/trips2.csv')
trips.head()

,Unnamed: 0,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,...,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income
0,0,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Otra vivienda,Centro estudios,Otros,...,1,2 adultos con niño(s),2,2,4,2,2,2,2021,12906.515
1,1,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Centro estudios,Compras doméstico-familiares,Estudio,...,1,2 adultos con niño(s),2,2,4,2,2,2,2021,12906.515
2,2,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Compras doméstico-familiares,Otra vivienda,Compras,...,1,2 adultos con niño(s),2,2,4,2,2,2,2021,12906.515
3,3,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,...,1,2 adultos con niño(s),1,5,1,1,2,1,2021,27765.010
4,4,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,25244.360


In [ ]:
# Eliminar los desplazamientos andando, no nos sirven para evaluar la movilidad de la region en su totalidad, pasa de 2M a 1,26M de desplazamientos

trips = trips.drop(trips.loc[trips['Modo'] == 'Andando'].index)

# Eliminar desplazamientos entre un mismo pueblo

trips = trips[trips['Mun_Ori'] != trips['Mun_Des']]
trips.head()

,Unnamed: 0,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,...,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income
3,3,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,...,1,2 adultos con niño(s),1,5,1,1,2,1,2021,27765.01
4,4,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,25244.36
9,9,20,20,Donostialdea,Urola Kosta,Andoain,Zarautz,Trabajo habitual,Gestiones personales,Trabajo,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,27765.01
13,13,20,20,Donostialdea,Donostialdea,Andoain,Donostia/San Sebastian,"Acompañamiento de ayuda, cuidado","Ocio, cultura, deporte",Gestiones Personales,...,1,Otros hogares sin niños,2,4,5,1,2,2,2021,15108.07
14,14,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Andoain,"Ocio, cultura, deporte",Residencia Habitual,Ocio,...,1,Otros hogares sin niños,2,4,5,1,2,2,2021,16703.26


In [ ]:
len(trips)

4649

In [ ]:
# Subo los pueblos con sus coordenadas

towns_coords = pd.read_excel('/content/drive/MyDrive/Code/Data/pueblos_gipuzkoa.xlsx')
columns_to_keep = ['Town','Latitud', 'Longitud']
towns_coords = towns_coords[columns_to_keep]
towns_coords

,Town,Latitud,Longitud
0,Abaltzisketa,43.051156,-2.110379
1,Aduna,43.214811,-2.054751
2,Aia,43.245113,-2.152427
3,Aizarnazabal,43.265408,-2.232831
4,Albiztur,43.128648,-2.143870
...,...,...,...
83,Zerain,43.010677,-2.284802
84,Zestoa,43.241819,-2.237987
85,Zizurkil,43.213223,-2.077485
86,Zumaia,43.287645,-2.246113


In [ ]:
# merged_df = trips.groupby(['Poblacion ORIGEN','Poblacion DESTINO','MODO_INFORME','MOTIVO_informe','Grupo edad','Actividad economica'])['Desplazamientos'].sum()
merged_df = trips.groupby(['Mun_Ori','Mun_Des'])['elev'].sum()
merged_df = pd.DataFrame(merged_df).reset_index()

merged_df = merged_df.merge(towns_coords, left_on='Mun_Ori', right_on='Town', how='left')

# Rename columns using the rename method
new_column_names = {'Latitud': 'Lat_origen', 'Longitud': 'Long_origen'}
merged_df.rename(columns=new_column_names, inplace=True)
column_to_delete = 'Town'
merged_df.drop(column_to_delete, axis=1, inplace=True)

merged_df = merged_df.merge(towns_coords, left_on='Mun_Des', right_on='Town', how='left')

# Rename columns using the rename method
new_column_names = {'Latitud': 'Lat_destino', 'Longitud': 'Long_destino'}
merged_df.rename(columns=new_column_names, inplace=True)
column_to_delete = 'Town'
merged_df.drop(column_to_delete, axis=1, inplace=True)

merged_df = merged_df.dropna()

merged_df

,Mun_Ori,Mun_Des,elev,Lat_origen,Long_origen,Lat_destino,Long_destino
0,Andoain,Beasain,704,43.219414,-2.020908,43.073261,-2.217907
1,Andoain,Donostia/San Sebastian,10960,43.219414,-2.020908,43.291811,-1.988513
2,Andoain,Errenteria,943,43.219414,-2.020908,43.255305,-1.892862
3,Andoain,Hernani,1576,43.219414,-2.020908,43.238783,-1.944259
4,Andoain,Hondarribia,344,43.219414,-2.020908,43.374885,-1.823971
...,...,...,...,...,...,...,...
419,Zumarraga,Oñati,514,43.103079,-2.315611,43.017826,-2.406793
420,Zumarraga,Tolosa,376,43.103079,-2.315611,43.107754,-2.080051
421,Zumarraga,Urretxu,1730,43.103079,-2.315611,43.099388,-2.324197
422,Zumarraga,Zarautz,229,43.103079,-2.315611,43.280963,-2.167224


# Modulo de distancias, traer el network de drive y calcular distancias entre pueblos

In [ ]:
# Crea un diccionario, donde los valores son None y las claves son esas cuatro

networks = dict.fromkeys({
 "drive",
})

# Asigna al valor correspondiente en el diccionario networks un objeto de red creado mediante el método from_hdf5() de la clase Network del módulo pdn.network.
# Los creados con los codigos anteriores

for k in networks:
    print(k)
    networks[k] = pdn.network.Network.from_hdf5('/content/drive/MyDrive/Code/MoCho_Gipuzkoa/networks/drive_net.h5')

drive


In [ ]:
# Calculo la distancia en coche entre los pueblos con el network creado para el MoCho

merged_df['distancia'] = networks['drive'].shortest_path_lengths(
        networks['drive'].get_node_ids(merged_df.Long_origen,merged_df.Lat_origen),
        networks['drive'].get_node_ids(merged_df.Long_destino,merged_df.Lat_destino),
        imp_name='distance'
        )
merged_df['distancia'] = merged_df['distancia']/1000 # pasar la distancia a kms
merged_df

,Mun_Ori,Mun_Des,elev,Lat_origen,Long_origen,Lat_destino,Long_destino,distancia
0,Andoain,Beasain,704,43.219414,-2.020908,43.073261,-2.217907,33.891003
1,Andoain,Donostia/San Sebastian,10960,43.219414,-2.020908,43.291811,-1.988513,9.905198
2,Andoain,Errenteria,943,43.219414,-2.020908,43.255305,-1.892862,15.519204
3,Andoain,Hernani,1576,43.219414,-2.020908,43.238783,-1.944259,10.381409
4,Andoain,Hondarribia,344,43.219414,-2.020908,43.374885,-1.823971,29.368208
...,...,...,...,...,...,...,...,...
419,Zumarraga,Oñati,514,43.103079,-2.315611,43.017826,-2.406793,19.075917
420,Zumarraga,Tolosa,376,43.103079,-2.315611,43.107754,-2.080051,27.453557
421,Zumarraga,Urretxu,1730,43.103079,-2.315611,43.099388,-2.324197,2.610198
422,Zumarraga,Zarautz,229,43.103079,-2.315611,43.280963,-2.167224,34.603958


# Cambiar el destino a uno más cercano

In [ ]:
# Hacer una lista de los pueblos que tiene cada pueblo disponible y eliminar columnas innecesarias

columnas_a_eliminar = ['elev', 'Lat_origen','Long_origen','Lat_destino','Long_destino']
distance_btw_towns = merged_df.drop(columns=columnas_a_eliminar)
distance_btw_towns

,Mun_Ori,Mun_Des,distancia
0,Andoain,Beasain,33.891003
1,Andoain,Donostia/San Sebastian,9.905198
2,Andoain,Errenteria,15.519204
3,Andoain,Hernani,10.381409
4,Andoain,Hondarribia,29.368208
...,...,...,...
419,Zumarraga,Oñati,19.075917
420,Zumarraga,Tolosa,27.453557
421,Zumarraga,Urretxu,2.610198
422,Zumarraga,Zarautz,34.603958


In [ ]:
# Salidas por pueblo calcula las salidas que tiene cada pueblo. Llegadas por pueblo calcula las llegadas.

# Ademas se expande y cada trayecto coge una fila.

llegadas_x_pueblo=merged_df.groupby(['Mun_Des'])['elev'].sum()
llegadas_x_pueblo = llegadas_x_pueblo.to_frame().reset_index()

salidas_x_pueblo=merged_df.groupby(['Mun_Ori'])['elev'].sum()
salidas_x_pueblo = salidas_x_pueblo.to_frame().reset_index()

# Duplicar filas según el número de salidas manteniendo el nombre del origen
salidas_x_pueblo = salidas_x_pueblo.loc[salidas_x_pueblo.index.repeat(salidas_x_pueblo['elev'])]
#salidas_x_pueblo['elev'] = 1  # Establecer el número de salidas en 1 para cada fila duplicada
salidas_x_pueblo=salidas_x_pueblo.reset_index()

# Eliminar la columna de desplazamientos

salidas_x_pueblo = salidas_x_pueblo.drop(columns=['elev'])

salidas_x_pueblo = salidas_x_pueblo['Mun_Ori']
salidas_x_pueblo=salidas_x_pueblo.to_frame()
salidas_x_pueblo

,Mun_Ori
0,Andoain
1,Andoain
2,Andoain
3,Andoain
4,Andoain
...,...
989063,Zumarraga
989064,Zumarraga
989065,Zumarraga
989066,Zumarraga


In [ ]:
# Voy a repartir las salidas de cada pueblo, con mayor probabilidad de que toque un pueblo cercano, y además con más llegadas.
# Entonces, a cada desplazamiento, primero le acorto las opciones diciendo que tiene que ser a un pueblo a menos de 15 km, y luego le asigno uno por probabilidad, dependiendo cuántas llegadas tienen esos pueblos destino

for i in range(len(salidas_x_pueblo)):
  origen = salidas_x_pueblo.loc[i, 'Mun_Ori']

  destinos_validos = distance_btw_towns[(distance_btw_towns['Mun_Ori'] == origen) & (distance_btw_towns['distancia'] < 15)]['Mun_Des']
  destinos_validos = destinos_validos.to_frame().reset_index()

  destinos_validos = destinos_validos['Mun_Des']
  destinos_validos=destinos_validos.to_frame()

  for j in range(len(destinos_validos)):
    for z in range(len(llegadas_x_pueblo)): #podria meter iterrows si no uso z
      valido=destinos_validos.loc[j,'Mun_Des']
      valor=llegadas_x_pueblo.loc[z,'Mun_Des']
      if valido == valor:
        d=llegadas_x_pueblo.loc[z,'elev']
        destinos_validos.loc[j,'Num']=d

  # Agregar la columna de probabilidad
  total_valores = destinos_validos['Num'].sum()

  # Calcular el porcentaje y agregarlo como una nueva columna
  destinos_validos['porcentaje'] = (destinos_validos['Num'] / total_valores) * 100

  #Calcular las probabilidades proporcionales a los porcentajes
  probabilidades = destinos_validos['porcentaje'] / 100

  # Realizar una selección ponderada basada en las probabilidades
  ciudad_seleccionada = random.choices(destinos_validos['Mun_Des'], weights=probabilidades)[0]

  salidas_x_pueblo.loc[i,'Mun_Des']=ciudad_seleccionada

salidas_x_pueblo


,Mun_Ori,Mun_Des
0,Andoain,Donostia/San Sebastian
1,Andoain,Lasarte-Oria
2,Andoain,Donostia/San Sebastian
3,Andoain,Donostia/San Sebastian
4,Andoain,Hernani
...,...,...
989063,Zumarraga,Urretxu
989064,Zumarraga,Urretxu
989065,Zumarraga,Bergara
989066,Zumarraga,Azkoitia


In [ ]:
merged_df = salidas_x_pueblo
merged_df

,Poblacion ORIGEN,Poblacion DESTINO,Desplazamientos
0,Andoain,Donostia/San Sebastian,1
1,Andoain,Donostia/San Sebastian,1
2,Andoain,Donostia/San Sebastian,1
3,Andoain,Usurbil,1
4,Andoain,Lasarte-Oria,1
...,...,...,...
444911,Zumarraga,Azkoitia,1
444912,Zumarraga,Bergara,1
444913,Zumarraga,Urretxu,1
444914,Zumarraga,Urretxu,1


In [ ]:
merged_df['Desplazamientos'] = 1
merged_df = merged_df.groupby(['Poblacion ORIGEN','Poblacion DESTINO'])['Desplazamientos'].sum()
merged_df = pd.DataFrame(merged_df).reset_index()
merged_df


# type(merged_df)

,Poblacion ORIGEN,Poblacion DESTINO,Desplazamientos
0,Andoain,Donostia/San Sebastian,5195
1,Andoain,Hernani,986
2,Andoain,Lasarte-Oria,1022
3,Andoain,Urnieta,386
4,Andoain,Usurbil,534
...,...,...,...
138,Zumaia,Zarautz,3639
139,Zumarraga,Azkoitia,2093
140,Zumarraga,Bergara,2595
141,Zumarraga,Legazpi,836


In [ ]:
# Incluir latitud y longitud a origen y a destino

merged_df = merged_df.merge(towns_coords, left_on='Poblacion ORIGEN', right_on='Town', how='left')

# Rename columns using the rename method
new_column_names = {'Latitud': 'Lat_origen', 'Longitud': 'Long_origen'}
merged_df.rename(columns=new_column_names, inplace=True)
column_to_delete = 'Town'
merged_df.drop(column_to_delete, axis=1, inplace=True)

merged_df = merged_df.merge(towns_coords, left_on='Poblacion DESTINO', right_on='Town', how='left')

# Rename columns using the rename method
new_column_names = {'Latitud': 'Lat_destino', 'Longitud': 'Long_destino'}
merged_df.rename(columns=new_column_names, inplace=True)
column_to_delete = 'Town'
merged_df.drop(column_to_delete, axis=1, inplace=True)

merged_df = merged_df.dropna()

columns_to_keep = ['Poblacion ORIGEN', 'Poblacion DESTINO', 'Desplazamientos', 'Lat_origen', 'Long_origen', 'Lat_destino', 'Long_destino']
merged_df = merged_df[columns_to_keep]
merged_df

,Poblacion ORIGEN,Poblacion DESTINO,Desplazamientos,Lat_origen,Long_origen,Lat_destino,Long_destino
0,Andoain,Donostia/San Sebastian,5195,43.219414,-2.020908,43.291811,-1.988513
1,Andoain,Hernani,986,43.219414,-2.020908,43.238783,-1.944259
2,Andoain,Lasarte-Oria,1022,43.219414,-2.020908,43.263787,-2.011931
3,Andoain,Urnieta,386,43.219414,-2.020908,43.227927,-1.964178
4,Andoain,Usurbil,534,43.219414,-2.020908,43.271458,-2.048138
...,...,...,...,...,...,...,...
138,Zumaia,Zarautz,3639,43.287645,-2.246113,43.280963,-2.167224
139,Zumarraga,Azkoitia,2093,43.103079,-2.315611,43.170649,-2.330963
140,Zumarraga,Bergara,2595,43.103079,-2.315611,43.122005,-2.414838
141,Zumarraga,Legazpi,836,43.103079,-2.315611,43.030754,-2.342367


In [ ]:
merged_df['Desplazamientos'].sum()

444916

In [ ]:
csv_path = '/content/drive/MyDrive/Code/Results/kepler_regiones_15_kms.csv'

# Guardar el DataFrame en un archivo CSV en la ubicación especificada
merged_df.to_csv(csv_path, index=False)

In [ ]:
# Calculo la distancia en coche entre los pueblos con el network creado para el MoCho

merged_df['distancia'] = networks['drive'].shortest_path_lengths(
        networks['drive'].get_node_ids(merged_df.Long_origen,merged_df.Lat_origen),
        networks['drive'].get_node_ids(merged_df.Long_destino,merged_df.Lat_destino),
        imp_name='distance'
        )
merged_df['distancia'] = merged_df['distancia']/1000 # pasar la distancia a kms
merged_df

,Poblacion ORIGEN,Poblacion DESTINO,Desplazamientos,Lat_origen,Long_origen,Lat_destino,Long_destino,distancia
0,Andoain,Donostia/San Sebastian,5195,43.219414,-2.020908,43.291811,-1.988513,9.905198
1,Andoain,Hernani,986,43.219414,-2.020908,43.238783,-1.944259,10.381409
2,Andoain,Lasarte-Oria,1022,43.219414,-2.020908,43.263787,-2.011931,5.696823
3,Andoain,Urnieta,386,43.219414,-2.020908,43.227927,-1.964178,6.422797
4,Andoain,Usurbil,534,43.219414,-2.020908,43.271458,-2.048138,8.954885
...,...,...,...,...,...,...,...,...
138,Zumaia,Zarautz,3639,43.287645,-2.246113,43.280963,-2.167224,10.401677
139,Zumarraga,Azkoitia,2093,43.103079,-2.315611,43.170649,-2.330963,10.108190
140,Zumarraga,Bergara,2595,43.103079,-2.315611,43.122005,-2.414838,13.042801
141,Zumarraga,Legazpi,836,43.103079,-2.315611,43.030754,-2.342367,9.240994


In [ ]:
merged_df['distancia_total'] = merged_df['distancia'] * merged_df['Desplazamientos']

In [ ]:
merged_df['distancia_total'].sum()

4434296.42379

# Seguir con el estudio

In [ ]:
# columns_to_keep = ['Poblacion ORIGEN', 'Poblacion DESTINO','MODO_INFORME','MOTIVO_informe','Grupo edad','Actividad economica','Desplazamientos', 'Lat_origen', 'Long_origen', 'Lat_destino', 'Long_destino','distancia]
columns_to_keep = ['Poblacion ORIGEN', 'Poblacion DESTINO','Desplazamientos', 'Lat_origen', 'Long_origen', 'Lat_destino', 'Long_destino','distancia']

merged_df = merged_df[columns_to_keep]
merged_df['distancia_total'] = merged_df['distancia'] * merged_df['Desplazamientos']
merged_df

,Poblacion ORIGEN,Poblacion DESTINO,Desplazamientos,Lat_origen,Long_origen,Lat_destino,Long_destino,distancia,distancia_total
9,Andoain,Beasain,499,43.219414,-2.020908,43.073261,-2.217907,33.891003,16911.610497
11,Andoain,Donostia/San Sebastian,2727,43.219414,-2.020908,43.291811,-1.988513,9.905198,27011.474946
12,Andoain,Errenteria,301,43.219414,-2.020908,43.255305,-1.892862,15.519204,4671.280404
14,Andoain,Hernani,764,43.219414,-2.020908,43.238783,-1.944259,10.381409,7931.396476
15,Andoain,Hondarribia,180,43.219414,-2.020908,43.374885,-1.823971,29.368208,5286.277440
...,...,...,...,...,...,...,...,...,...
773,Zumarraga,Errenteria,258,43.103079,-2.315611,43.255305,-1.892862,55.506398,14320.650684
775,Zumarraga,Legazpi,842,43.103079,-2.315611,43.030754,-2.342367,9.240994,7780.916948
776,Zumarraga,Ordizia,233,43.103079,-2.315611,43.063758,-2.181832,16.787698,3911.533634
779,Zumarraga,Urretxu,1737,43.103079,-2.315611,43.099388,-2.324197,2.610198,4533.913926


In [ ]:
merged_df['distancia_total'].sum()

7672674.234813

In [ ]:
merged_df['Desplazamientos'].sum()

444916

In [ ]:
csv_path = '/content/drive/MyDrive/Code/Results/kepler_situacion_actual.csv'

# Guardar el DataFrame en un archivo CSV en la ubicación especificada
merged_df.to_csv(csv_path, index=False)